In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0))

CUDA Available: True
Device Count: 2
GPU Name: Tesla T4


In [2]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)

print("HF Login Successful")

HF Login Successful


In [3]:
from transformers import pipeline
import torch

MODEL_ID = "google/medgemma-4b-it"

pipe = pipeline(
    "text-generation",
    model=MODEL_ID,
    device=0,
    torch_dtype=torch.bfloat16
)

print("Model loaded successfully")

config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Model loaded successfully


In [4]:
system_prompt = """
You are a rural healthcare triage assistant.

Your role:
- Analyze patient symptoms and vitals.
- Do NOT diagnose.
- Do NOT prescribe medication.
- Classify risk level: Low, Moderate, or High.
- Identify red flag symptoms.
- Suggest whether immediate referral is needed.
- Highlight missing important information.

Return STRICT JSON in this format:

{
  "summary": "...",
  "risk_level": "...",
  "red_flags": [],
  "recommended_action": "...",
  "missing_information": [],
  "safety_note": "This is not a medical diagnosis."
}

Return JSON only.
"""

In [5]:
import json

def clean_json_output(text):
    if not text or len(text.strip()) == 0:
        raise ValueError("Empty model output")

    text = text.replace("json", "").replace("", "").strip()

    start = text.find("{")
    end = text.rfind("}") + 1

    if start == -1 or end == -1:
        raise ValueError("No JSON object found in output")

    json_str = text[start:end]
    return json.loads(json_str)

In [6]:
def safety_override(parsed_output, patient_input):

    symptoms = [s.lower() for s in patient_input.get("symptoms", [])]
    vitals = patient_input.get("vitals", {})
    age = patient_input.get("age", 0)

    bp = vitals.get("blood_pressure", "")
    systolic = None
    diastolic = None
    try:
        systolic = int(bp.split("/")[0])
        diastolic = int(bp.split("/")[1])
    except:
        pass

    hr = vitals.get("heart_rate", "")
    heart_rate = None
    try:
        heart_rate = int(hr.split()[0])
    except:
        pass

    # ----------------------------
    # RULE 1: Hypertensive Crisis
    # ----------------------------
    if systolic and systolic >= 180:
        parsed_output["risk_level"] = "High"
        parsed_output["recommended_action"] = "Immediate emergency evaluation required due to hypertensive crisis."

    # ----------------------------
    # RULE 2: Shock Risk
    # ----------------------------
    if systolic and systolic < 90:
        parsed_output["risk_level"] = "High"
        parsed_output["recommended_action"] = "Urgent evaluation required due to possible shock."

    # ----------------------------
    # RULE 3: Severe Tachycardia
    # ----------------------------
    if heart_rate and heart_rate > 130:
        parsed_output["risk_level"] = "High"
        parsed_output["recommended_action"] = "Immediate evaluation required due to severe tachycardia."

    # ----------------------------
    # RULE 4: Stroke Suspicion
    # ----------------------------
    stroke_keywords = ["slurred speech", "weakness", "numbness", "confusion", "blurred vision"]
    if any(keyword in symptoms for keyword in stroke_keywords):
        if age >= 50:
            parsed_output["risk_level"] = "High"
            parsed_output["recommended_action"] = "Immediate stroke evaluation recommended."

    # ----------------------------
    # RULE 5: Cardiac Risk
    # ----------------------------
    chest_keywords = ["chest pain", "chest discomfort", "pressure in chest"]
    if age >= 60 and any(keyword in symptoms for keyword in chest_keywords):
        parsed_output["risk_level"] = "High"
        parsed_output["recommended_action"] = "Immediate cardiac emergency referral recommended."

    # ----------------------------
    # RULE 6: Unconsciousness
    # ----------------------------
    if "unconsciousness" in symptoms:
        parsed_output["risk_level"] = "High"
        parsed_output["recommended_action"] = "Immediate emergency referral required."

    return parsed_output

In [7]:
import json

patient_input = {
    "age": 52,
    "gender": "Female",
    "symptoms": ["Severe headache", "Blurred vision"],
    "duration": "1 day",
    "vitals": {
        "blood_pressure": "180/110",
        "heart_rate": "95 bpm"
    }
}

user_prompt = f"Patient Data:\n{json.dumps(patient_input, indent=2)}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = pipe(messages, max_new_tokens=600, do_sample=False)

raw_output = response[0]["generated_text"][-1]["content"]

parsed_output = clean_json_output(raw_output)
final_output = safety_override(parsed_output, patient_input)

print(json.dumps(final_output, indent=2))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{
  "summary": "Patient presents with a severe headache and blurred vision for one day, accompanied by elevated blood pressure (180/110) and slightly elevated heart rate (95 bpm).",
  "risk_level": "High",
  "red_flags": [
    "Severe headache",
    "Blurred vision",
    "Elevated blood pressure (180/110)",
    "Elevated heart rate (95 bpm)"
  ],
  "recommended_action": "Immediate stroke evaluation recommended.",
  "missing_information": [
    "Pain location and characteristics of the headache",
    "Associated symptoms (e.g., nausea, vomiting, neck stiffness, weakness, numbness, speech difficulties)",
    "Medical history (e.g., hypertension, diabetes, heart disease, stroke history)",
    "Medications",
    "Allergies",
    "Recent trauma or illness",
    "Family history of stroke or hypertension"
  ],
  "safety_note": "This is not a medical diagnosis."
}


In [8]:
import time
import pandas as pd

evaluation_cases = [
    {"age": 22, "gender": "Female", "symptoms": ["Mild cough"], "duration": "3 days",
     "vitals": {"blood_pressure": "118/76", "heart_rate": "70 bpm"}},

    {"age": 30, "gender": "Male", "symptoms": ["Mild headache"], "duration": "1 day",
     "vitals": {"blood_pressure": "120/80", "heart_rate": "72 bpm"}},

    {"age": 45, "gender": "Female", "symptoms": ["Persistent cough", "Fever"],
     "duration": "5 days",
     "vitals": {"blood_pressure": "130/85", "heart_rate": "90 bpm"}},

    {"age": 60, "gender": "Male", "symptoms": ["Chest pain", "Shortness of breath"],
     "duration": "30 minutes",
     "vitals": {"blood_pressure": "160/95", "heart_rate": "110 bpm"}},

    {"age": 28, "gender": "Male", "symptoms": ["Unconsciousness"],
     "duration": "10 minutes",
     "vitals": {"blood_pressure": "100/60", "heart_rate": "120 bpm"}},
]

evaluation_cases = evaluation_cases * 5
evaluation_cases = evaluation_cases[:25]

results = []
override_count = 0
risk_counts = {"Low": 0, "Moderate": 0, "High": 0}

start_time = time.time()

for case in evaluation_cases:
    user_prompt = f"Patient Data:\n{json.dumps(case, indent=2)}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = pipe(messages, max_new_tokens=600, do_sample=False)
    raw_output = response[0]["generated_text"][-1]["content"]

    try:
        parsed_output = clean_json_output(raw_output)
    except:
        continue

    before = parsed_output["risk_level"]
    final_output = safety_override(parsed_output, case)
    after = final_output["risk_level"]

    if before != after:
        override_count += 1

    risk_counts[after] += 1

    results.append({
        "Age": case["age"],
        "Symptoms": ", ".join(case["symptoms"]),
        "Before": before,
        "Final": after
    })

end_time = time.time()

avg_latency = round((end_time - start_time) / len(results), 2)

df = pd.DataFrame(results)

print("===== EVALUATION SUMMARY =====")
print("Cases:", len(results))
print("Overrides:", override_count)
print("Override %:", round((override_count / len(results)) * 100, 2))
print("Avg Latency:", avg_latency)
print("Risk Distribution:", risk_counts)

df.head()

Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

===== EVALUATION SUMMARY =====
Cases: 25
Overrides: 0
Override %: 0.0
Avg Latency: 15.91
Risk Distribution: {'Low': 10, 'Moderate': 5, 'High': 10}


,Age,Symptoms,Before,Final
0,22,Mild cough,Low,Low
1,30,Mild headache,Low,Low
2,45,"Persistent cough, Fever",Moderate,Moderate
3,60,"Chest pain, Shortness of breath",High,High
4,28,Unconsciousness,High,High


In [9]:
import time
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# Evaluation Cases WITH Expected Risk Label
# ==========================================

evaluation_cases = [
    # Low Risk
    {"age": 22, "gender": "Female", "symptoms": ["Mild cough"],
     "duration": "3 days",
     "vitals": {"blood_pressure": "118/76", "heart_rate": "70 bpm"},
     "expected_risk": "Low"},

    {"age": 30, "gender": "Male", "symptoms": ["Mild headache"],
     "duration": "1 day",
     "vitals": {"blood_pressure": "120/80", "heart_rate": "72 bpm"},
     "expected_risk": "Low"},

    # Moderate Risk
    {"age": 45, "gender": "Female", "symptoms": ["Persistent cough", "Fever"],
     "duration": "5 days",
     "vitals": {"blood_pressure": "130/85", "heart_rate": "90 bpm"},
     "expected_risk": "Moderate"},

    # High Risk
    {"age": 60, "gender": "Male", "symptoms": ["Chest pain", "Shortness of breath"],
     "duration": "30 minutes",
     "vitals": {"blood_pressure": "160/95", "heart_rate": "110 bpm"},
     "expected_risk": "High"},

    {"age": 28, "gender": "Male", "symptoms": ["Unconsciousness"],
     "duration": "10 minutes",
     "vitals": {"blood_pressure": "100/60", "heart_rate": "120 bpm"},
     "expected_risk": "High"},
]

# Expand to 25 cases
evaluation_cases = evaluation_cases * 5
evaluation_cases = evaluation_cases[:25]

# ==========================================
# Run Evaluation
# ==========================================

results = []
true_labels = []
pred_labels = []

start_time = time.time()

for case in evaluation_cases:

    user_prompt = f"Patient Data:\n{json.dumps(case, indent=2)}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = pipe(messages, max_new_tokens=600, do_sample=False)
    raw_output = response[0]["generated_text"][-1]["content"]

    try:
        parsed_output = clean_json_output(raw_output)
    except:
        continue

    final_output = safety_override(parsed_output, case)

    true_labels.append(case["expected_risk"])
    pred_labels.append(final_output["risk_level"])

    results.append({
        "Age": case["age"],
        "Symptoms": ", ".join(case["symptoms"]),
        "Expected": case["expected_risk"],
        "Predicted": final_output["risk_level"]
    })

end_time = time.time()

# ==========================================
# Metrics
# ==========================================

df = pd.DataFrame(results)

accuracy = sum([1 for i in range(len(true_labels)) if true_labels[i] == pred_labels[i]]) / len(true_labels)

conf_matrix = confusion_matrix(true_labels, pred_labels, labels=["Low","Moderate","High"])

avg_latency = round((end_time - start_time) / len(true_labels), 2)

print("=========================================")
print("CLINICAL SAFETY EVALUATION")
print("=========================================")
print("Total Cases:", len(true_labels))
print("Accuracy:", round(accuracy * 100, 2), "%")
print("Average Inference Time (sec):", avg_latency)
print("\nConfusion Matrix (Low, Moderate, High):")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))

df.head()

Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=600) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

CLINICAL SAFETY EVALUATION
Total Cases: 25
Accuracy: 100.0 %
Average Inference Time (sec): 13.33

Confusion Matrix (Low, Moderate, High):
[[10  0  0]
 [ 0  5  0]
 [ 0  0 10]]

Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00        10
         Low       1.00      1.00      1.00        10
    Moderate       1.00      1.00      1.00         5

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25



,Age,Symptoms,Expected,Predicted
0,22,Mild cough,Low,Low
1,30,Mild headache,Low,Low
2,45,"Persistent cough, Fever",Moderate,Moderate
3,60,"Chest pain, Shortness of breath",High,High
4,28,Unconsciousness,High,High


In [10]:
import gradio as gr
import json
import time

# -----------------------------
# Confidence Estimation
# -----------------------------
def compute_confidence(risk_level, overridden):
    if overridden:
        return 0.95
    if risk_level == "High":
        return 0.85
    if risk_level == "Moderate":
        return 0.70
    return 0.60


# -----------------------------
# Main Triage Function (HARDENED)
# -----------------------------
def triage_interface(age, gender, symptoms, duration, bp, hr):
    try:
        # -----------------------------
        # Input Structuring
        # -----------------------------
        patient_input = {
            "age": int(age),
            "gender": gender,
            "symptoms": [s.strip() for s in symptoms.split(",") if s.strip()],
            "duration": duration,
            "vitals": {
                "blood_pressure": bp,
                "heart_rate": hr
            }
        }

        user_prompt = f"Patient Data:\n{json.dumps(patient_input, indent=2)}"

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        start = time.time()

        # -----------------------------
        # Model Inference
        # -----------------------------
        response = pipe(
            messages,
            max_new_tokens=400,
            do_sample=False,
            truncation=True
        )

        raw_output = response[0]["generated_text"][-1]["content"]

        # -----------------------------
        # Safe JSON Parsing
        # -----------------------------
        try:
            parsed_output = clean_json_output(raw_output)
        except Exception:
            return f"""
### ⚠️ Model Output Error

The model did not return valid structured JSON.

Raw Output:
{raw_output}
"""

        # -----------------------------
        # Safety Override
        # -----------------------------
        before = parsed_output.get("risk_level", "Unknown")

        final_output = safety_override(parsed_output, patient_input)

        after = final_output.get("risk_level", "Unknown")

        overridden = before != after
        confidence = compute_confidence(after, overridden)

        latency = round(time.time() - start, 2)

        # -----------------------------
        # Risk Display Formatting
        # -----------------------------
        if after == "High":
            risk_display = "🔴 HIGH RISK"
        elif after == "Moderate":
            risk_display = "🟡 MODERATE RISK"
        else:
            risk_display = "🟢 LOW RISK"

        # Safe key access
        summary = final_output.get("summary", "Not available")
        action = final_output.get("recommended_action", "Not available")
        red_flags = ", ".join(final_output.get("red_flags", []))
        missing = ", ".join(final_output.get("missing_information", []))

        result = f"""
### {risk_display}

*Summary:*  
{summary}

*Recommended Action:*  
{action}

*Red Flags:*  
{red_flags}

*Missing Information:*  
{missing}

---

*Confidence Score:* {round(confidence * 100, 1)}%  
*Inference Time:* {latency} seconds  
*Safety Override Triggered:* {overridden}

---

SafeTriage: Offline Hybrid AI Escalation for Community Health Workers  
Powered by MedGemma + Deterministic Safety Layer
"""

        return result

    except Exception as e:
        return f"""
### ❌ Internal Processing Error

An unexpected error occurred.

Details:
{str(e)}

Please verify inputs and try again.
"""


# -----------------------------
# Gradio UI
# -----------------------------
demo = gr.Interface(
    fn=triage_interface,
    inputs=[
        gr.Number(label="Age"),
        gr.Dropdown(["Male", "Female", "Other"], label="Gender"),
        gr.Textbox(label="Symptoms (comma separated)"),
        gr.Textbox(label="Duration"),
        gr.Textbox(label="Blood Pressure (e.g., 120/80)"),
        gr.Textbox(label="Heart Rate (e.g., 72 bpm)")
    ],
    outputs=gr.Markdown(label="Triage Assessment"),
    title="🩺 SafeTriage",
    description="Offline Hybrid AI Escalation Assistant for Community Health Workers",
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://439d31dc459a210b72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
# ==========================================
# SAFETY STRESS TESTING
# ==========================================

stress_cases = [
    # Borderline chest pain younger patient
    {"age": 35, "gender": "Male", 
     "symptoms": ["Mild chest discomfort"], 
     "duration": "2 hours",
     "vitals": {"blood_pressure": "125/80", "heart_rate": "85 bpm"},
     "expected_risk": "Moderate"},

    # Headache without hypertension
    {"age": 40, "gender": "Female", 
     "symptoms": ["Headache"], 
     "duration": "1 day",
     "vitals": {"blood_pressure": "120/80", "heart_rate": "72 bpm"},
     "expected_risk": "Low"},

    # Fever + tachycardia
    {"age": 50, "gender": "Male", 
     "symptoms": ["Fever", "Weakness"], 
     "duration": "2 days",
     "vitals": {"blood_pressure": "110/70", "heart_rate": "125 bpm"},
     "expected_risk": "Moderate"},

    # Elderly mild fatigue
    {"age": 75, "gender": "Female", 
     "symptoms": ["Fatigue"], 
     "duration": "3 days",
     "vitals": {"blood_pressure": "130/85", "heart_rate": "80 bpm"},
     "expected_risk": "Moderate"},

    # Stroke-like symptom mild case
    {"age": 55, "gender": "Male", 
     "symptoms": ["Blurred vision"], 
     "duration": "30 minutes",
     "vitals": {"blood_pressure": "150/90", "heart_rate": "88 bpm"},
     "expected_risk": "High"},
]

true_labels = []
pred_labels = []

for case in stress_cases:
    user_prompt = f"Patient Data:\n{json.dumps(case, indent=2)}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = pipe(messages, max_new_tokens=400, do_sample=False)
    raw_output = response[0]["generated_text"][-1]["content"]

    parsed_output = clean_json_output(raw_output)
    final_output = safety_override(parsed_output, case)

    true_labels.append(case["expected_risk"])
    pred_labels.append(final_output["risk_level"])

print("Stress Test Results:")
for i in range(len(stress_cases)):
    print(f"Expected: {true_labels[i]} | Predicted: {pred_labels[i]}")

Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

Stress Test Results:
Expected: Moderate | Predicted: Moderate
Expected: Low | Predicted: Low
Expected: Moderate | Predicted: High
Expected: Moderate | Predicted: Moderate
Expected: High | Predicted: High


In [12]:
false_negatives = 0
false_positives = 0

for i in range(len(true_labels)):
    if true_labels[i] == "High" and pred_labels[i] != "High":
        false_negatives += 1
    if true_labels[i] != "High" and pred_labels[i] == "High":
        false_positives += 1

print("Safety Analysis Summary")
print("False Negatives (High missed):", false_negatives)
print("False Positives (Over-escalation):", false_positives)

Safety Analysis Summary
False Negatives (High missed): 0
False Positives (Over-escalation): 1
